# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
output_data_file = "../output_data/weather_data.csv"
weather_data_df = pd.read_csv(output_data_file)
weather_data_df

,City,Lat,Lng,Cloudiness,Date,Humidity,Max Temp,Wind Speed,Country
0,alice springs,-23.70,133.88,0,1592435696,51,37.40,4.70,AU
1,castro,-24.79,-50.01,0,1592435696,93,56.70,4.43,BR
2,diffa,13.32,12.61,98,1592435696,52,83.84,14.58,NE
3,butajira,8.12,38.37,77,1592435697,78,60.60,1.90,ET
4,shu,43.60,73.76,1,1592435697,68,54.59,4.45,KZ
...,...,...,...,...,...,...,...,...,...
545,sovetskaya,44.78,41.17,0,1592435810,8,62.01,4.00,RU
546,frontera,26.93,-101.45,75,1592435811,38,96.01,9.17,MX
547,orocue,4.79,-71.34,75,1592435811,90,77.49,5.57,CO
548,aksay,47.26,39.87,20,1592435811,56,73.00,4.47,RU


In [ ]:
# weather_data = pd.read_csv("weather_data.csv") 
# weather_data.head()
# weather_data.read_csv("../output_data/weather_data.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Plot Heatmap
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"].astype(float)

# Create heat layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=np.max(humidity),
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

weather_conditions_df = weather_data_df.loc[(weather_data_df["Max Temp"] > 70) & (weather_data_df["Max Temp"] < 80) &
                                           (weather_data_df["Cloudiness"] == 0) & (weather_data_df["Wind Speed"] < 10)]
weather_conditions_df = weather_conditions_df.dropna(how='any')
weather_conditions_df.reset_index(inplace=True)
del weather_conditions_df['index']
weather_conditions_df

,City,Lat,Lng,Cloudiness,Date,Humidity,Max Temp,Wind Speed,Country
0,namibe,-15.20,12.15,0,1592435712,60,71.44,6.55,AO
1,marmaris,36.85,28.27,0,1592435718,93,73.40,3.36,TR
2,gat,31.61,34.76,0,1592435735,45,75.99,1.01,IL
3,awbari,26.59,12.78,0,1592435740,16,77.32,7.05,LY
4,turayf,31.67,38.66,0,1592435780,36,75.20,9.17,SA
5,gonbad-e qabus,37.25,55.17,0,1592435782,64,73.40,2.24,IR
6,ierapetra,35.01,25.74,0,1592435797,69,71.01,1.99,GR


In [32]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# weather_conditions_df = pd.DataFrame(weather_data_df, columns = ["Cloudiness", "Humidity", "Max Temp", "Wind Speed"])
# weather_max_temp = (weather_conditions_df["Max Temp"] <= 80) & (weather_conditions_df["Max Temp"] >= 70)
# weather_wind_speed = weather_conditions_df["Wind Speed"] < 10
# weather_cloudiness = weather_conditions_df["Cloudiness"] == 0

# Storing variables
# weather_clean_df = weather_conditions_df[weather_max_temp & weather_wind_speed & weather_cloudiness]
# weather_clean_df = weather_conditions_df.dropna()
# weather_clean_df.reset_index(inplace=True)
# del weather_clean_df['index']
# weather_clean_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
# Create hotel_df dataframe
hotel_df = weather_conditions_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,namibe,AO,-15.20,12.15,
1,marmaris,TR,36.85,28.27,
2,gat,IL,31.61,34.76,
3,awbari,LY,26.59,12.78,
4,turayf,SA,31.67,38.66,
5,gonbad-e qabus,IR,37.25,55.17,
6,ierapetra,GR,35.01,25.74,


In [53]:
# Create open list
hotel_name = []

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {"location": target_coordinates,
          "type": target_type,
          "keyword": target_search,
          "radius": target_radius,
          "key": "g_key"}


for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    # Set Parameters
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Print Results
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
 
    # Get Results
    response['results']
    
    # Find closest hotel and store in dataframe
    try:
        print(f"The Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_name.append(results[results][0]['name'])
    
    # If there isn't a hotel, report missing
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: namibe.
Missing field/result... skipping.
Retrieving Results for Index 1: marmaris.
Missing field/result... skipping.
Retrieving Results for Index 2: gat.
Missing field/result... skipping.
Retrieving Results for Index 3: awbari.
Missing field/result... skipping.
Retrieving Results for Index 4: turayf.
Missing field/result... skipping.
Retrieving Results for Index 5: gonbad-e qabus.
Missing field/result... skipping.
Retrieving Results for Index 6: ierapetra.
Missing field/result... skipping.


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig